In [1]:
import pickle
import pandas as pd
import sklearn
import numpy as np
import scipy
# load the dic file

from utils import *

path = 'cleaned/module_dict.pickle'
with open(path, 'rb') as f:
    module_dict = pickle.load(f)

tissues = ['islet', 'liver', 'adipose', 'kidney', 'gastroc']
signs = ['SIGNED', 'UNSIGNED']

X_train, X_test, Y_train, Y_test = get_data('islet','SIGNED','red')

# print the shape of training / testing
X_train.shape,X_test.shape,Y_train.shape,Y_test.shape

((441, 764), (50, 764), (441, 4), (50, 4))

In [3]:
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF, ConstantKernel as C, Matern, RationalQuadratic, DotProduct
from sklearn.model_selection import GridSearchCV

from sklearn.preprocessing import StandardScaler

# Define the kernel for Gaussian Process

# Initialize the scaler
scaler_X = StandardScaler().fit(X_train)

# Apply the transformation
X_train_norm = scaler_X.transform(X_train)
X_test_norm = scaler_X.transform(X_test)

In [22]:



# Radial basis function (RBF) kernel
kernel_rbf = C(1.0, (1e-5, 1e3)) * RBF(length_scale_bounds = (1e-5, 1e2))

# Matérn kernel with different gamma & length-scale parameter
# The Matern kernel's nu parameter is equivalent to its smoothness. Higher values make it more similar to the RBF kernel. 
# Let's use nu = [0.5, 1.5, 2.5, 3.5] to capture different degrees of smoothness.
kernels_matern = [C(1.0, (1e-5, 1e3)) * Matern(length_scale_bounds= (1e-5, 1e2), nu= n) for n in [0.5, 1.5, 2.5]]

# Rational quadratic kernel with different alpha (scale mixture parameter) & length-scale parameters
kernel_rq = C(1.0, (1e-5, 1e3)) * RationalQuadratic(alpha_bounds=(1e-5, 1e3), length_scale_bounds=(1e-5, 1e2))

# Dot-Product kernel (we can combine it with exponential kernel, but for simplicity, let's just define it for now)
kernel_dotproduct = C(1.0, (1e-5, 1e3)) * DotProduct(sigma_0_bounds=(1e-5, 1e2))

all_kernels = [kernel_rbf] + kernels_matern + [kernel_rq, kernel_dotproduct]

param_grid = {
    "kernel": all_kernels,
    "alpha": [0.1]}

# Initialize GP Regressor
model = GaussianProcessRegressor(n_restarts_optimizer=1,normalize_y=True,random_state=0)

# Initialize GridSearchCV
gsh = GridSearchCV(estimator=model, param_grid=param_grid, cv=3, scoring='neg_mean_squared_error',verbose = 1,n_jobs=-1)  # or another suitable scoring method

gsh.fit(X_train, Y_train)

# Best model
best_gp = gsh.best_estimator_

# Predictions
Y_pred = best_gp.predict(X_test)

Fitting 3 folds for each of 6 candidates, totalling 18 fits


In [6]:
# visualize the gsh.cv_results_
gsh.cv_results_

{'mean_fit_time': array([ 2.81688819,  3.48648472, 12.7392971 ,  2.68929672,  2.98889132,
         5.1936305 ,  6.42164869,  1.51656566,  3.15341759,  3.74115667,
         7.66977491,  2.88051481,  4.9195364 ,  5.14299197,  5.76226034,
         2.77930937,  3.67322326,  4.97261324,  6.17258501,  2.73676047,
         4.18425183,  4.55506968, 13.75398393,  2.58870177]),
 'std_fit_time': array([0.60072254, 0.44581943, 5.21006613, 0.76424466, 1.0406368 ,
        1.67568046, 1.68578322, 0.12773951, 0.79516354, 1.01488913,
        2.56645157, 0.16014535, 0.93582493, 1.2067051 , 2.03723931,
        0.13410357, 0.64970736, 0.78725622, 1.25593428, 0.11610897,
        1.73951219, 1.16670954, 2.5270979 , 0.08369337]),
 'mean_score_time': array([0.02125425, 0.02103095, 0.02974672, 0.00811858, 0.02343907,
        0.02320442, 0.02042432, 0.00482073, 0.0217833 , 0.02130485,
        0.02430539, 0.00823188, 0.02854357, 0.02730689, 0.02758126,
        0.00822463, 0.02814889, 0.02736082, 0.02760239, 0.00

In [14]:
for i in gsh.cv_results_['rank_test_score']:
    print(gsh.cv_results_['params'][i-1])

{'alpha': 0.01, 'kernel': 1**2 * DotProduct(sigma_0=1)}
{'alpha': 0.0001, 'kernel': 1**2 * DotProduct(sigma_0=1)}
{'alpha': 1e-06, 'kernel': 1**2 * Matern(length_scale=1, nu=1.5)}
{'alpha': 1, 'kernel': 1**2 * RationalQuadratic(alpha=0.1, length_scale=1)}
{'alpha': 0.1, 'kernel': 1**2 * RBF(length_scale=1)}
{'alpha': 0.0001, 'kernel': 1**2 * RationalQuadratic(alpha=0.1, length_scale=1)}
{'alpha': 1e-06, 'kernel': 1**2 * RBF(length_scale=1)}
{'alpha': 1, 'kernel': 1**2 * Matern(length_scale=1, nu=1.5)}
{'alpha': 0.01, 'kernel': 1**2 * RationalQuadratic(alpha=0.1, length_scale=1)}
{'alpha': 0.0001, 'kernel': 1**2 * Matern(length_scale=1, nu=1.5)}
{'alpha': 1e-18, 'kernel': 1**2 * DotProduct(sigma_0=1)}
{'alpha': 1, 'kernel': 1**2 * RBF(length_scale=1)}
{'alpha': 0.01, 'kernel': 1**2 * Matern(length_scale=1, nu=1.5)}
{'alpha': 0.0001, 'kernel': 1**2 * RBF(length_scale=1)}
{'alpha': 1e-18, 'kernel': 1**2 * RationalQuadratic(alpha=0.1, length_scale=1)}
{'alpha': 0.1, 'kernel': 1**2 * DotPro

In [8]:
# compute the average mean percentage error
def mean_percentage_error(y_true, y_pred):
    return np.mean(np.abs((y_true - y_pred) / y_true),axis=0) * 100

per_err = mean_percentage_error(Y_test, Y_pred)
# compute the mean percentage error
per_err_mean = np.mean(per_err,axis=0)
# merge per_err and per_err_mean as pandas dataframe
per_err['mean'] = per_err_mean
per_err

glucose         18.619099
weight           8.595634
insulin         80.065468
triglyceride    34.983085
mean            35.565821
dtype: float64